In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\adity\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
ext_data = load_pdf("Data/")

In [5]:
def text_split(ext_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(ext_data)

    return text_chunks

In [6]:
text_chunks = text_split(ext_data)

In [7]:
print(len(text_chunks))

7020


In [8]:
# #download embedding model
# def download_hugging_face_embeddings():
#     embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     return embeddings
# embeddings = download_hugging_face_embeddings()

In [9]:
from dotenv import load_dotenv
load_dotenv()
import os

In [10]:
PINECONE1_API_KEY = os.environ.get('PINECONE1_API_KEY')


In [11]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE1_API_KEY)
index = pc.Index("medicalchatbot")

In [12]:
from sentence_transformers import SentenceTransformer

# Function to download the Hugging Face embedding model
def download_hugging_face_embeddings():
    embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Load the Hugging Face embedding model
model = download_hugging_face_embeddings()

In [13]:


# ### Upserting data with meta data
# # Extract the data from text chunks

# cleaned_data = [t.page_content for t in text_chunks][2000:3000]

# # Generate embeddings for each text chunk
# embeddings = model.encode(cleaned_data)

# # Formatting for upserting

# entries = []
# for i, embedding in enumerate(embeddings):
#     entry = {
#         "id": f"text-{i+3000}",  # Replace with your ID format
#         "values": embedding.tolist(),
#         "metadata": {"context":cleaned_data[i]}  # Assuming metadata is provided in the same order as embeddings
#     }
#     entries.append(entry)

# index.upsert(vectors=entries)

 Querying

In [67]:
query_text = "acne"


In [68]:
import torch
import numpy as np

# Assuming you have already embedded and stored embeddings for this query text
# Generate the embedding for the query text (you would typically do this in the embedding step)
query_embedding = model.encode([query_text])[0]

# Ensure query embedding is a list of floats
if isinstance(query_embedding, np.ndarray):
    query_embedding = query_embedding.tolist()
elif isinstance(query_embedding, torch.Tensor):
    query_embedding = query_embedding.tolist()
else:
    raise ValueError(f"Unexpected embedding type: {type(query_embedding)}")



In [69]:
# Search for similar embeddings
results = index.query(vector=query_embedding, top_k=4,include_metadata=True)

In [70]:
final_result = [result['metadata']['context'] for result in results['matches']]
context = ((" ".join(final_result)).replace('\n','')).strip()
context


'GALE ENCYCLOPEDIA OF MEDICINE 2 25AcneAcne vulgaris affecting a woman’s face. Acne is the generalname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-ates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25 ent purposes. For example, lotions, soaps, gels, andcreams containing benzoyl peroxide or tretinoin may beused to clear up mild to moderately severe acne.Isotretinoin (Accutane) is prescribed only for verysevere, disfiguring acne.Acne is a skin condition that occurs when pores or Cliffs, NJ: Prentice Hall, 1995.Goldstein, Sanford M., and Richard B. Odom. “Skin &Appendages: Pustular Disorders.” In Current MedicalDiagnosis and Treatment, 1996. 35th ed. Ed. StephenMcPhee, et al. Stamford: Appleton & Lange, 1995.Kaptchuk, Ted J., Z’ev Rosenberg, and K’an Herb Co., Inc.K’an Herbals: Formulas by Ted Kaptchuk, O.M.D. SanFrancisco: Andrew Miller, 1996.PERIODICALS“Adult Acne.” Harvard Wome

In [71]:
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate

prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
# chain_type_kwargs = {"prompt": PROMPT}

# Load the model
llm = CTransformers(model="Model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens': 128,
                            'temperature': 0.3 })




# Define the user's question
question = "What is acne and how it is treated? "

# Format the prompt
formatted_prompt = PROMPT.format(context=context, question=question)



In [72]:
print(formatted_prompt)


Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: GALE ENCYCLOPEDIA OF MEDICINE 2 25AcneAcne vulgaris affecting a woman’s face. Acne is the generalname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-ates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25 ent purposes. For example, lotions, soaps, gels, andcreams containing benzoyl peroxide or tretinoin may beused to clear up mild to moderately severe acne.Isotretinoin (Accutane) is prescribed only for verysevere, disfiguring acne.Acne is a skin condition that occurs when pores or Cliffs, NJ: Prentice Hall, 1995.Goldstein, Sanford M., and Richard B. Odom. “Skin &Appendages: Pustular Disorders.” In Current MedicalDiagnosis and Treatment, 1996. 35th ed. Ed. StephenMcPhee, et al. Stamford: Appleton & Lange, 1995.Ka